https://phdinds-aim.github.io/time_series_handbook/08_WinningestMethods/lightgbm_m5_forecasting.html

https://www.kaggle.com/code/ratan123/m5-forecasting-lightgbm-with-timeseries-splits

In [7]:
import pandas as pd
import numpy as np
import gc

In [8]:
import lightgbm as lgb
from sklearn import metrics

In [9]:
#TODO in fine tuning
params = {'num_leaves': 555,
          'min_child_weight': 0.034,
          'feature_fraction': 0.379,
          'bagging_fraction': 0.418,
          'min_data_in_leaf': 106,
          'objective': 'regression',
          'max_depth': -1,
          'learning_rate': 0.005,
          "boosting_type": "gbdt",
          "bagging_seed": 11,
          "metric": 'rmse',
          "verbosity": -1,
          'reg_alpha': 0.3899,
          'reg_lambda': 0.648,
          'random_state': 222,
         }

In [ ]:
### read data
train_df = pd.read_csv("../data/preprocessed_train_df.csv")
test_df = pd.read_csv("../data/preprocessed_test_df.csv")

In [ ]:
import joblib
id_cols = ['Client', 'Warehouse', 'Product']

In [ ]:
### for forecast_horizon create separate dataframes for training and prediction
columns = ['holiday_count', 'is_holiday', 'is_prev_1week_holidays', 'is_prev_2week_holidays',
           'year', 'month', 'week', 'day', 'day_of_week', 'quarter']

predictions = []

for horizon, date in enumerate(test_df['ds'].unique()):
    horizon_specific_cols = ['sales_lag_ph_{i}' for i in range(horizon, 14+10)] +\
                            ['price_lag__ph_{i}' for i in range(horizon, 14+10)] +\
                            ['price_rolling_mean_ph_{i}' for i in range(horizon, 14+10)] +\
                            ['price_rolling_std_ph_{i}' for i in range(horizon, 14+10)] +\
                            ['sales_rolling_std_ph_{i}' for i in range(horizon, 14+10)] +\
                            ['sales_rolling_mean_ph_{i}' for i in range(horizon, 14+10)]
    
    x_train = train_df[columns+horizon_specific_cols].copy()
    y_train = train_df['y'].copy()

    x_pred = test_df[test_df['ds']==date][columns+horizon_specific_cols].copy()

    pred_combo = test_df[id_cols+['ds']]

    clf = lgb.train(params, [x_train, y_train], 2500)

    joblib.dump(f"../model/vn1_horizon_{horizon}".pkl)

    pred_combo['prediction'] = clf.predict(x_pred)
    predictions.append(pred_combo)

    del x_train, y_train, x_pred
    gc.collect()
    

In [ ]:
predictions_df = pd.concat(predictions)

In [ ]:
# dates = test_df['ds'].unique()
# predictions_df = dict(zip(dates, predictions))

# predictions_df = pd.DataFrame(predictions_df)

In [ ]:
actual_sales = pd.read_csv("../data/actual_sales_p2.csv")
actual_sales['ds'] = pd.to_datetime(actual_sales['ds'] )

In [ ]:
out =actual_sales.merge(predictions_df, on = [id_cols+['ds']], how = 'inner')

In [ ]:
from eval import custom_error

cerr_mod = custom_error(out['y'], out['prediction'])
print('  LightGBM Error For all Horizons: %.4f' % cerr_mod)